In [1]:
from IPython.display import HTML

import math
import pandas as pd
import numpy as np

#from scipy.optimize import minimize
#from scipy.optimize import differential_evolution    

from lmfit import Minimizer, Parameters, report_fit

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import LinearAxis, Range1d
from bokeh.models import DatetimeTickFormatter, MonthsTicker, NumeralTickFormatter, Legend

output_notebook()

Loading BokehJS ...

In [2]:
import sys
sys.path.insert(1,'../covid-website')

from data import Data


In [3]:
source = 'Johns Hopkins'    
region = 'US'
state = 'Florida'

cutoff_positive = 1
cutoff_death = 1
truncate = 0
window = 14

d = Data(source=source, region=region, state=state, county="", cutoff_positive=cutoff_positive, cutoff_death=cutoff_death, truncate=truncate) 


In [4]:
h = d.hospitalization()



In [5]:
e = d.excess_deaths()


In [6]:

#find the time period when we have both excess deaths data and covid fatalities data

ti0 = pd.to_datetime(e['start_date'], format='%Y-%m-%d')
ti1 = pd.to_datetime(e['end_date'], format='%Y-%m-%d')
t0 = np.min(ti0)
t1 = np.max(ti1)

t0 = max(t0, min(d.xd))
t1 = min(t1, max(d.xd))

#find excess deaths for this time period
mask = (ti0 >= t0) & (ti1 <= t1)
excess = np.sum(e[mask]['excess_deaths'])

#find the covid deaths for this time period
mask = (d.xd >= t0) & (d.xd <= t1)
f0 = d.fatalities[mask][0]
f1 = d.fatalities[mask][-1]
covid = (f1-f0)

print(t0, t1, excess, covid, excess/covid)



2020-03-03 00:00:00 2021-02-06 00:00:00 34790.50125516625 27599 1.2605710806611201


In [23]:
x = d.x[d.minD+1:]
y = d.dfatalities
y = np.nan_to_num(y)
print(y)

p = figure(y_axis_type="log")
p.line(x,y)
show(p)

[  0.   0.   0.   0.   0.   0.   1.   0.   2.   2.   2.   2.   2.   0.
   5.   0.   5.   6.   6.  19.   2.   7.  22.   2.  77.   6.  25.  26.
  15.  47.  26.  45.  36.  48.  23.  38.  72.  25.  72.  57.  23.  26.
  48.  45.  26.  94.  59.   9.  20.  13.  83.  47.  50.  46.  50.  15.
  20.  72.  68.  61.  69.  46.   6.  14.  44.  48.  48.  42.  47.   9.
  24.  55.  44.  48.  46.  43.   4.  15.   7.  60.  45.  49.  34.   4.
   9.  70.  36.  41.  53.  28.  12.  12.  53.  36.  47.  29.  48.   6.
   7.  55.  25.  43.  43.  40.  17.  12.  65.  43.  46.  39.  24.  29.
  28.  58.  45.  67.  67.  18.  29.  47.  63.  48. 120.  93.  95.  45.
  35. 132. 112. 156. 128.  90.  87.  90. 134. 139. 173. 135. 124.  77.
  77. 186. 216. 253. 257. 179.  62.  73. 245. 225. 120. 180. 182.  77.
  91. 276. 212. 148. 228. 204. 107.  87. 219. 174. 117. 119. 106.  51.
  72. 183. 153. 135.  89. 148.  14.  68. 187. 127. 149. 100.  61.  38.
  22.  44. 195. 216. 176.  98.   8.  34. 145. 152. 147. 139.  62.   9.
  21. 

In [11]:
p = figure(title='Daily', plot_width=800, plot_height=600 , y_axis_type="log")
p.y_range.start = 1

p.line(d.xd[d.minD+1:], d.dfatalities, line_width=1, line_color='red', line_dash='solid', alpha=0.3)
p.circle(d.xd[d.minD+1:], d.dfatalities, color='red', alpha=0.3)

p.line(e['date'], e['excess_deaths']/7, line_width=1, line_color='blue', line_dash='solid', alpha=1)



p.yaxis[0].formatter = NumeralTickFormatter(format="0,0")
p.xaxis.ticker = MonthsTicker(months=list(range(1,13)))
p.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

p.xgrid.ticker = p.xaxis.ticker
p.xaxis.major_label_orientation = math.pi/4

p.extra_x_ranges['x2'] = Range1d(d.x[0], d.x[-1])
ax2 = LinearAxis(x_range_name="x2", axis_label="days")
p.add_layout(ax2, 'below')


p.ygrid.grid_line_color = 'navy'
p.ygrid.grid_line_alpha = 0.3
p.ygrid.minor_grid_line_color = 'navy'
p.ygrid.minor_grid_line_alpha = 0.1

p.yaxis[0].axis_label = 'Number of deaths per day'

show(p)

In [16]:
p = figure(title='Daily', plot_width=800, plot_height=600 , y_axis_type="log")
p.y_range.start = 1

p.x_range.start = d.xd.iloc[0]
p.x_range.end = d.xd.iloc[-1]


p.extra_x_ranges['x2'] = Range1d(d.x[0], d.x[-1])
ax2 = LinearAxis(x_range_name="x2", axis_label="days")
p.add_layout(ax2, 'below')


p.line(d.xd[d.minD+1:], d.dfatalities, line_width=1, line_color='red', line_dash='solid', alpha=0.3)
p.circle(d.xd[d.minD+1:], d.dfatalities, color='red', alpha=0.3)

p.line(e['date'], e['excess_deaths']/7, line_width=1, line_color='blue', line_dash='solid', alpha=1)

p.line(d.x[d.minD+1:], d.dfatalities, line_width=1, line_color='green', line_dash='solid', alpha=0.3, x_range_name='x2' )


p.yaxis[0].formatter = NumeralTickFormatter(format="0,0")
p.xaxis[0].ticker = MonthsTicker(months=list(range(1,13)))
p.xaxis[0].formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

p.xgrid.ticker = p.xaxis[0].ticker  #[0] needed because we added a second x-axis earlier
p.xaxis[0].major_label_orientation = math.pi/4



p.ygrid.grid_line_color = 'navy'
p.ygrid.grid_line_alpha = 0.3
p.ygrid.minor_grid_line_color = 'navy'
p.ygrid.minor_grid_line_alpha = 0.1

p.yaxis[0].axis_label = 'Number of deaths per day'

show(p)